In [2]:
from langchain.document_loaders import UnstructuredURLLoader

loader = UnstructuredURLLoader(
    urls=[
        "https://www.theverge.com/24054854/tesla-next-generation-vehicle-ev-elon-musk",
        "https://economictimes.indiatimes.com/stocksupdate/companyid-12934.cms"
    ]
)

data = loader.load()
type(data)


list

In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n', '.', ' '],
    chunk_size=500,
    chunk_overlap=0
)

chunks = []

for document in data:
    chunks.extend(splitter.split_text(document.page_content))

len(chunks)

99

In [5]:
from langchain.embeddings import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(openai_api_key="your api key")

In [7]:
from langchain.vectorstores import Chroma

collection = Chroma(embedding_function=embedding_model, collection_name="article-vectors")

for chunk in chunks:
    collection.add_texts([chunk])

In [23]:
query = "where was rebel attack on tesla and how it affected tesla?"

# Perform the search
search_type = "similarity"  # You can also use "mmr" or "similarity_score_threshold"
results = collection.search(query, search_type=search_type, k=5)  # k=5 for top 5 results
type(results)
# print("Search Results:")
# for doc in results:
#     print("************************************")
#     print(doc.page_content)  


list

In [30]:
from openai import OpenAI

client = OpenAI(
    api_key='your api key',
)

responses = []

for doc in results:
    chunk_text = doc.page_content 

    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"{chunk_text} \n\n Based on this chunk, can you provide a summary or answer the question '{query}'?"
            }
        ],
        model="gpt-3.5-turbo"
    )
    
    responses.append(response.choices[0].message.content)

final_prompt = "\n".join(responses)

final_res = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": f"{final_prompt} \n\n Based on the information above, can you provide a final answer or summary for the question '{query}'?"
        }
    ],
    model="gpt-3.5-turbo"
)
    
print(final_res.choices[0].message.content)

The rebel attack on Tesla occurred in the Red Sea, causing shipping delays for the company. This, along with other production challenges, contributed to a decline in sales growth for Tesla. Additionally, the attack may have influenced CEO Elon Musk's decision to cancel plans for a more affordable electric vehicle and focus on developing an autonomous vehicle instead.
